## QR法

### 上 Hessenberg 化

上 Hessenberg 化是指将矩阵通过相似变换变成形如如下的 上 Hessenberg 矩阵 (上三角 + 下对角线):
$$Q^*AQ = H = \left[\begin{matrix}h_{11}  &  h_{12} &  \dotsc  &  h_{1,n-1} &  h_{1n} \\
 h_{21} &  h_{22}  &  \dotsc  &  h_{2,n-1} &  h_{2n} \\
\ &  h_{32}  &  \dotsc  &  h_{3,n-1}  &  h_{3n} \\
\ &   & \ddots &\vdots & \vdots\\
\ &   &   &  h_{n,n-1}  &  h_{nn}\end{matrix}\right].$$

这个可以通过 Householder 变换完成.

### 

In [2]:
import numpy as np
import math
from numba import jit

@jit(nopython=True)
def HouseholderVector(A):
    squarelength = np.dot(A[1:].conj(), A[1:])
    if squarelength == 0: # 已经完成了,不需要做变换
        return A, A, 1

    # 对于复向量 x, 需要确定向量 y, 使得 y 的第一个分量模长等于 x 的范数
    # 且 y,x 的内积是实数
    # 才有 Householder变换 Qx = y
    normal     = A.copy()
    rotation   = A[0]/abs(A[0]) if A[0] != 0 else 1. 
    normal[0] = (abs(A[0]) + math.sqrt(max((squarelength + A[0] * A[0].conjugate()).real,0))) * rotation

    # 用返回两个不同模长的向量替代正规化
    normal2 = normal.conj() * ( -2. /(squarelength + normal[0] * normal[0].conjugate()))
    
    return normal, normal2, 0

# Householder 作用在矩阵左侧
@jit(nopython=True)
def leftHouse(A,vectors):
    normal , normal2 , flag = vectors
    if flag:
        return 
    tmp = normal2 @ A
    n = tmp.shape[0]
    # 这里做原地修改而非生成 n*n 矩阵才能不损失效率 , 亦即 CGEMM
    # 为了使用 for 循环, 额外用到 jit 加速, 这样做比 numpy 生成 n*n 矩阵后再修改快很多倍
    for i in range(normal.shape[0]):
        for j in range(n):
            A[i,j] += normal[i]*tmp[j]

# Householder 作用在矩阵右侧
@jit(nopython=True)
def rightHouse(A,vectors):
    normal , normal2 , flag = vectors
    if flag:
        return 
    tmp = A @ normal
    n = normal2.shape[0]
    for i in range(tmp.shape[0]):
        for j in range(n):
            A[i,j] += tmp[i]*normal2[j]

In [7]:
@jit(nopython=True)
def Hessenberg(A,copy = 1):
    if copy: A = A.copy()
    n = A.shape[0] 
    Q = np.eye(n,dtype=A.dtype)
    for i in range(n-1):
        v = HouseholderVector(A[i+1:,i])
        leftHouse(A[i+1:,i:],v)
        A[i+2:,i] = 0
        rightHouse(Q[:,i+1:],v)
        rightHouse(A[:,i+1:],v)
    return (Q,A)

n = 5
A = np.random.randn(n*n).reshape((n,n)) #+ np.random.randn(n*n).reshape((n,n))*1j
Q , H = Hessenberg(A)
print(H)
print('||Q*AQ - H|| =',np.linalg.norm(Q.T.conj() @ A @ Q - H))

[[-0.05195029  0.75262492  0.20593938 -0.11062057  1.30025321]
 [-2.74198784  0.82685427  0.21978669 -0.84176976 -0.4044097 ]
 [ 0.          1.8882943   1.61118514  0.12324276 -0.57050253]
 [ 0.          0.         -1.48703306 -0.25455821  0.27532976]
 [ 0.          0.          0.          1.38553814 -0.27263681]]
||Q*AQ - H|| = 1.2927037611685434e-15


## 对称特征值

实对称矩阵 / Hermite 矩阵的特征值是一类重要的问题. 若已知矩阵是实对称 / Hermite 的, 那么利用这个信息可能产生更快的算法. 此外, 矩阵 $A$ 的奇异值分解可以转化为对称特征值问题 —— 求解$A^*A$ 的特征值. 


### Jacobi 对角化

矩阵 $A$ 是实对称矩阵. 选取 $A$ 模长最大的非对角元(之一) $a_{ij} = \overline {a_{ji}}$. 一对作用在第 $i,j$ 行与第 $i,j$ 列的左右 Givens 变换将 $a_{ij},a_{ji}$ 变成零. 重复该过程, 则最终 $A$ 趋近于对角阵. 

$$\left[\begin{matrix}c & s\\ -s & c\end{matrix}\right]
\left[\begin{matrix}a_{ii} & a_{ij}\\ a_{ij} & a_{jj}\end{matrix}\right]
\left[\begin{matrix}c & -s\\ s & c\end{matrix}\right] = 
\left[\begin{matrix}a_{ii}' & 0\\ 0 & a_{jj}'\end{matrix}\right]$$

比对右上角元素, 
$$(c^2 - s^2)a_{ij} + cs(a_{ii} - a_{jj}) = 0.$$

令 $\tau = \frac{a_{ii}-a_{jj}}{a_{ij}}$, $t = \frac{s}{c}$, 则 $t^2+2\tau t -1 = 0$. 

我们取定 $t$ 为绝对值较小的根, 数学上已证明这有利于提高收敛速度, 即 
$$t = \frac{{\rm sgn}(\tau)}{|\tau| + \sqrt{1+\tau ^2}}.$$

以及 $c = \frac{1}{\sqrt{1+t^2}}$, $s = ct$.



In [1]:
@jit(nopython=True)
def Givens(a,b,angle):
    cost , sint = angle
    tmp = cost * a + sint * b
    b[:] = (-sint).conjugate() * a + cost * b
    a[:] = tmp

@jit(nopython = True)
def JacobiDiagonalization(A, iter = -1, copy = 1):
    n = A.shape[0]
    if iter < 0: iter = 5 * n * n
    if copy: A = A.copy()
    Q = np.eye(n,dtype=A.dtype)
    
    for _ in range(iter):
        # 找到模长最大的 A[x][y]
        x , y = 0, 1
        for i in range(n):
            for j in range(i+1,n):
                if abs(A[i,j]) > abs(A[x,y]):
                    x , y = i , j

        if abs(A[x,y]) < (abs(A[x,x]) + abs(A[y,y])) * 2e-16: 
            # 如果最大值也很小, 则收敛
            break

        coeff = (A[x,x] - A[y,y]) * 0.5 /A[x,y]
        tant = 1./(abs(coeff) + math.sqrt(1+coeff*coeff))
        if coeff < 0: tant = -tant
        cost = 1./math.sqrt(1 + tant*tant)
        sint = cost * tant
        angle = (cost, sint)

        Givens(A[x],A[y],angle)
        Givens(Q[x],Q[y],angle)
        Givens(A[:,x],A[:,y],angle)
        
    return Q , A

n = 20
np.random.seed(1)
A = np.random.randn(n*n).reshape((n,n))
A = A.T @ A
Q , D = JacobiDiagonalization(A)
print('Orthogonality Loss  =',np.linalg.norm(Q.T @ Q - np.eye(n)))
print('Q.T @ D @ Q - A     =',np.linalg.norm(Q.T @ D @ Q - A))

# 观察一下 D 的非对角元的部分
V = D.copy()
for i in range(n):
    V[i,i] = 0
print('Non Diagonal Entries =',np.linalg.norm(V))

#print(sorted([D[i,i] for i in range(n)]))
#print(sorted(np.linalg.eigvalsh(A)))

Orthogonality Loss  = 1.0444528553297399e-14
Q.T @ D @ Q - A     = 4.837182546420556e-13
Non Diagonal Entries = 3.490912071133264e-14


注: 若 $A$ 是 Hermite 矩阵则该方法不可行.
$$\left[\begin{matrix}c & s\\ -\overline s & c\end{matrix}\right]
\left[\begin{matrix}a_{ii} & a_{ij}\\ \overline {a_{ij}} & a_{jj}\end{matrix}\right]
\left[\begin{matrix}c & -\overline s\\ s & c\end{matrix}\right] = 
\left[\begin{matrix}a_{ii}' & 0\\ 0 & a_{jj}'\end{matrix}\right]$$

比对右上角元素, 
$$c^2a_{ij} - |s|^2\overline{a_{ij}} + c\overline s(a_{ii} - a_{jj}) = 0.$$


注意 $a_{ii},a_{jj},c\in \mathbb R$, 若记 $s = |s|e^{i\phi}, t = \frac{|s|}{|c|}$, 则

$$t^2\overline{a_{ij}} - e^{-i\phi}(a_{ii} - a_{jj})t - a_{ij}  = 0.$$

设 $t_1t_2 = \frac{-a_{ij}}{\overline {a_{ij}}} = -e^{2i\alpha}$, 则
$$\left|\frac{a_{ii}-a_{jj}}{a_{ij}}\right| = |t_1+t_2| = |t_1 - \frac{\cos 2\alpha}{t_1} - \frac{i\sin 2\alpha}{t_1}|$$
$$t_1^4-(2\cos 2\alpha +\left|\frac{a_{ii}-a_{jj}}{a_{ij}}\right|^2)t_1^2 +1 = 0 $$
当 $\Delta < 0$, 方程无实根.

### 循环 Jacobi (Cyclic Jacobi)
上述 Jacobi 有一大效率的缺陷——每次都要花 $O(n^2)$ 的时间找到最大的非对角元, 但 Givens 变换只需要 $O(n)$ 的时间. 循环 Jacobi 方法提出可以依次选取非对角元 $(1,2),(1,3),\dotsc,(1,n),(2,3),(2,4),\dotsc,(n-1,n)$. 即每个非对角元都做一遍, 多来几轮. Wilkinson 证明了这是二次收敛的.

In [11]:
import numpy as np
import math 
from numba import jit

@jit(nopython=True)
def Givens(a,b,angle):
    cost , sint = angle
    tmp = cost * a + sint * b
    b[:] = (-sint).conjugate() * a + cost * b
    a[:] = tmp

@jit(nopython = True)
def JacobiDiagonalizationCyclic(A, iter = -1, copy = 1):
    n = A.shape[0]
    if iter < 0: iter = 5 * n * n
    if copy: A = A.copy()
    Q = np.eye(n,dtype=A.dtype)
    
    x, y = 0, 1
    for _ in range(iter):
        # 找到模长最大的 A[x][y]
        y += 1
        if y == n:
            x , y = x + 1 , x + 2
            if x == n - 1:
                x , y = 0 , 1
                
        if abs(A[x,y]) < (abs(A[x,x]) + abs(A[y,y])) * 2e-16: 
            # 如果最大值也很小, 则收敛
            break

        coeff = (A[x,x] - A[y,y]) * 0.5 /A[x,y]
        tant = 1./(abs(coeff) + math.sqrt(1+coeff*coeff))
        if coeff < 0: tant = -tant
        cost = 1./math.sqrt(1 + tant*tant)
        sint = cost * tant
        angle = (cost, sint)

        Givens(A[x],A[y],angle)
        Givens(Q[x],Q[y],angle)
        Givens(A[:,x],A[:,y],angle)
        
    return Q , A

n = 100
np.random.seed(1)
A = np.random.randn(n*n).reshape((n,n))
A = A.T @ A

# 比较一下效率
%timeit JacobiDiagonalization(A)
%timeit JacobiDiagonalizationCyclic(A)

338 ms ± 37.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
53.1 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 双对角化 (Golub-Kahan Bidiagonalization)

欲计算 $A$ 的奇异值分解, 当然可以先计算 $A^*A$, 然后再 Hessenberg 化变成三对角矩阵， 但也可以按如下方法进行:
找酉阵 $U$, $V$, 使得 $UAV^*$ 是形如如下的双对角阵:
$$UAV^*=\left[\begin{matrix}\times &\times &  & & \\
\ & \times & \times & & \\
\ &  & \ddots & \ddots & \\
\ & & & \times& \times\\
\ & & &  & \times\end{matrix}\right]$$

那么 $VA^*AV^* = (VA^*U^*)(UAV^*)$ 自然是三对角阵.

找到 $U$ 和 $V$ 可以通过 Householder 变换实现: 先用左 Householder 把第一列的下面置零, 再用右 Householder 把第一行置零. 接下来处理第二行, 第二列 ...

另外, Chan [2] 指出, 若 $A\in \mathbb C^{m\times n}$ 且 $m>\frac{5}{3}n$, 可以先用 QR 分解把下面 $(m-n)\times n$ 的部分变成零, 再对上面的 $n\times n$ 部分做双对角化, 这样可以减少运算量.

双对角化完毕后并不一定要乘出三对角再运用 QR 算法求奇异值分解, 见 [1] pp. 489.

## References

1. G. Golub and C. Van Loan, Matrix Computations, The Johns Hopkins University Press, Baltimore, Maryland, 4th ed., 2013.
2. Chan, Tony F. , 
    An Improved Algorithm for Computing the Singular Value Decomposition.
    , Acm Transactions on Mathematical Software, **8.1(1982)**, 72--83.